# True Shower Regression Baseline

Create a baseline using a simple ML algorithm to predict/infer the true shower parameters using a simple ML algorithm
- using clean_image_\*\_m1 and clean_image_\*\_m2 independently
- by combining both clean_image_* features with "hillas" and/or "stereo"

Linear regression, polynomial regression, decision tree regression and random forrest regression will be used.

In [2]:
import pandas as pd
import torch
from torch.utils.data import random_split
import matplotlib.pyplot as plt
import numpy as np
import pyarrow.parquet as pq

import math
import sys
import os

sys.path.append("../../../magic-ml-images")
from magicdl import magic

sys.path.append("../..")

SEED = 42
gen = torch.Generator().manual_seed(SEED)

## Load and prepare data

In [3]:
gammas = pd.read_parquet("../../../data/magic-gammas.parquet")
protons = pd.read_parquet("../../../data/magic-protons.parquet")

In [4]:
gammas = gammas.dropna()
protons = protons.dropna()

In [5]:
NUM_PIXELS = 1183

ts_params = [
    "true_energy", 
    "true_theta",
    "true_phi", 
    "true_telescope_theta", 
    "true_telescope_phi",
    "true_first_interaction_height",
    "true_impact_m1",
    "true_impact_m2"
]
hillas_params = [
    "hillas_length_m1",
    "hillas_width_m1",
    "hillas_delta_m1",
    "hillas_size_m1",
    "hillas_cog_x_m1",
    "hillas_cog_y_m1",
    "hillas_sin_delta_m1",
    "hillas_cos_delta_m1",
    "hillas_length_m2",
    "hillas_width_m2",
    "hillas_delta_m2",
    "hillas_size_m2",
    "hillas_cog_x_m2",
    "hillas_cog_y_m2",
    "hillas_sin_delta_m2",
    "hillas_cos_delta_m2"
]
stereo_params = [
    "stereo_direction_x",       
    "stereo_direction_y",       
    "stereo_zenith",            
    "stereo_azimuth",           
    "stereo_dec",               
    "stereo_ra",                
    "stereo_theta2",            
    "stereo_core_x",            
    "stereo_core_y",            
    "stereo_impact_m1",         
    "stereo_impact_m2",         
    "stereo_impact_azimuth_m1", 
    "stereo_impact_azimuth_m2", 
    "stereo_shower_max_height", 
    "stereo_xmax",              
    "stereo_cherenkov_radius",  
    "stereo_cherenkov_density", 
    "stereo_baseline_phi_m1",   
    "stereo_baseline_phi_m2",   
    "stereo_image_angle",   
    "stereo_cos_between_shower"
]

# split the dataset:

num_protons = len(protons)
num_gammas = len(gammas)

TRAIN_PORTION = 0.6
TEST_PORTION = 0.2
# VAL_PORTION is the rest

train_size_protons = int(TRAIN_PORTION * num_protons)
test_size_protons = int(TEST_PORTION * num_protons)
val_size_protons = num_protons - train_size_protons - test_size_protons

train_size_gammas = int(TRAIN_PORTION * num_gammas)
test_size_gammas = int(TEST_PORTION * num_gammas)
val_size_gammas = num_gammas - train_size_gammas - test_size_gammas

protons_indices = torch.randperm(num_protons, generator=gen)
gammas_indices = torch.randperm(num_gammas, generator=gen)

protons_train_indices = protons_indices[:train_size_protons]
protons_val_indices = protons_indices[train_size_protons:train_size_protons+val_size_protons]
protons_test_indices = protons_indices[train_size_protons+val_size_protons:]

gammas_train_indices = gammas_indices[:train_size_gammas]
gammas_val_indices = gammas_indices[train_size_gammas:train_size_gammas+val_size_gammas]
gammas_test_indices = gammas_indices[train_size_gammas+val_size_gammas:]

protons_train = protons.iloc[protons_train_indices]
protons_val = protons.iloc[protons_val_indices]
protons_test = protons.iloc[protons_test_indices]
gammas_train = gammas.iloc[gammas_train_indices]
gammas_val = gammas.iloc[gammas_val_indices]
gammas_test = gammas.iloc[gammas_test_indices]

# protons_ts = protons[ts_params]
# gammas_ts = gammas[ts_params]
# protons_clean1 = protons["clean_image_m1"]
# gammas_clean1 = gammas["clean_image_m1"]
# protons_clean2 = protons["clean_image_m2"]
# gammas_clean2 = gammas["clean_image_m2"]
# protons_hillas = protons[hillas_params]
# gammas_hillas = gammas[hillas_params]
# protons_stereo = protons[stereo_params]
# gammas_stereo = gammas[stereo_params]

y_protons_train = protons_train[ts_params]
y_protons_val = protons_val[ts_params]
y_protons_test = protons_test[ts_params]
y_gammas_train = gammas_train[ts_params]
y_gammas_val = gammas_val[ts_params]
y_gammas_test = gammas_test[ts_params]

# normalise data:

from sklearn.preprocessing import StandardScaler

def normalise(train, val, test):
    scaler = StandardScaler()
    train_new = scaler.fit_transform(train.copy())
    val_new = scaler.transform(val)
    test_new = scaler.transform(test)
    return (train_new, val_new, test_new), scaler

(y_protons_train, y_protons_val, y_protons_test), y_proton_scaler = normalise(y_protons_train, y_protons_val, y_protons_test)

(y_gammas_train, y_gammas_val, y_gammas_test), y_gamma_scaler = normalise(y_gammas_train, y_gammas_val, y_gammas_test)


## Using linear regression

In [6]:
import linear_regression
from linear_regression import LinearRegression
import importlib
importlib.reload(linear_regression)

<module 'linear_regression' from '/home/teame/ML_Team_E/src/infer_true_shower_parameters/baseline/linear_regression.py'>

### Training only on clean_image_m1
#### Protons

In [7]:
y_train = torch.tensor(y_protons_train, dtype=torch.float32)
y_val = torch.tensor(y_protons_val, dtype=torch.float32)
y_test = torch.tensor(y_protons_test, dtype=torch.float32)

X_train = np.array(protons_train["clean_image_m1"].tolist())
X_val = np.array(protons_val["clean_image_m1"].tolist())
X_test = np.array(protons_test["clean_image_m1"].tolist())

(X_train, X_val, X_test), _ = normalise(X_train, X_val, X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

In [9]:
linreg = LinearRegression(NUM_PIXELS)

linreg.fit(X_train, y_train, lr=0.0001)

y_pred_val = linreg.predict(X_val)
y_pred_train = linreg.predict(X_train)


/home/teame/ML_Team_E/src/infer_true_shower_parameters/baseline/linear_regression.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
/home/teame/ML_Team_E/src/infer_true_shower_parameters/baseline/linear_regression.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)
/home/teame/ML_Team_E/src/infer_true_shower_parameters/baseline/linear_regression.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)


In [11]:
mse_val = np.mean((y_pred_val - y_val.numpy())**2)
mse_train = np.mean((y_pred_train - y_train.numpy())**2)
print((mse_train, mse_val))

# ss_res = np.sum((y_pred - y_val.numpy())**2)
# ss_tot = np.sum((y_val.numpy() - np.mean(y_val.numpy(), axis=0))**2)
# r2_score = 1 - (ss_res / ss_tot)
# print(f"R^2 Score: {r2_score}")

(np.float32(1.061619), np.float32(1.056821))
